# What is happening at the heat of LLMs?

Before transformers, recurrent neural networks (RNNs) were considered the cutting edge in Natural Language Processing (NLP). An RNN is a type of neural network where outputs from previous steps are fed as inputs to the current step. This characteristic enables an RNN to retain information from previous steps, making them well-suited for sequential data like text. In the context of NLP, an RNN takes an input, such as a word or character, processes it through its network, and generates a vector known as the hidden state. If you are unfamiliar with RNNs, don't worry, you don't need to know the detailed workings of RNNs to follow this discussion. 

One area where RNNs played an important role was in the development of machine translation systems, where the model translates text from one language to another. However, the word sequence in one language might be different from another one due to the grammatical structures in the source and target language. To address this issue we can use an encoder-decoder architecture. The encoder's role is to convert input sequence information into a numerical representation, typically referred to as the final hidden state. The encoder updates its hidden state at each step, trying to capture the entire meaning of the input sentence in the final hidden state. The decoder then takes this final hidden state to start generating the translated sentence, one word at a time. \
However, a significant challenge of this architecture lies in the fact that the final hidden state of the encoder creates an information bottleneck. it has to represent the meaning of the whole input sequence because this is all the decoder has access to when generating the output. This is especially challenging for long sequences, where information at the start of the sequence might be lost in the process of compressing everything to a single, fixed representation.

To address this challenge, an "attention mechanism" is introduced, permitting the decoder to selectively access different hidden states of the encoder. But, why selective? Using all the states at the same time would create a huge input for the decoder, the attention mechanism lets the decoder assign a different amount of weight, or "attention" to each of the encoder states at every decoding timestep. \
Researchers, as detailed in the paper "Attention is all you need," have demonstrated that RNN architectures are not required for NLP applications such as machine translation and proposed a transformer architecture with a “self-attention mechanism”.  

The main idea behind the self-attention mechanism is that instead of using fixed embeddings for each token, we can use the whole sequence to compute a weighted average of each embedding. Given a sequence of token embeddings $ x_{1}, ..., x_{n} $ self-attention produces a sequence of new embeddings $ x_{1}^{'}, ..., x_{n}^{'} $ where each $ x_{i}^{'} $ is a linear combination of all the $ x_{j}^{'},  j=1...n $: 

$(1) \; x_{i} = \sum \limits _{j=1} ^{n} w_{ji}x_{j}$


With this introduction, let's dive into implementation.

In [ ]:
! pip install torch==2.0.1

In [6]:
import torch
from torch import nn

Imagine we have an embedding model that generates embeddings in a 6 dimensional embedding space. Assume that our embedding model has generated the following embedding vectors for our input sentence "Write your first Attention mechanism".

Please note that embedding values in this example are totally random and don't express any information.

In [7]:
# Write your first Attention mechanism
input_embeddings = torch.tensor(
  [[0.172, 0.295, 0.618, 0.459, 0.818, 0.071], # Write 
   [0.265, 0.563, 0.718, 0.323, 0.126, 0.235], # your                                               
   [0.206, 0.333, 0.044, 0.862, 0.152, 0.594], # first    
   [0.300, 0.505, 0.727, 0.495, 0.898, 0.954], # Attention     
   [0.095, 0.809, 0.596, 0.110, 0.447, 0.418]] # mechanism   
)

In [8]:
print(input_embeddings.shape)

torch.Size([5, 6])


# Positional Encoding
Positional encoding augments the token embeddings by adding position-dependent information about the tokens in a sequence. To this end, we add "positional encodings" to the input embeddings at the bottoms of the encoder and decoder stacks. 

An easy solution could be to start token positions from 0 and continue until all tokens are counted. However, this solution may end up with very large numbers in a large sequence of tokens. Instead, a positional encoding vector is created for each position, meaning a positional encoding matrix can be created to represent all the possible positions a word can take. 

There are many choices of positional encodings, the authors in [1] used the sine and cosine functions to generate a unique vector for each position in the sequence. 


$ (2) \; PE(pos, 2i) = sin(\frac {pos}{10000^{\frac{2i}{d_{e}}}}), \; PE(pos, 2i+1) = cos(\frac {pos}{10000^{\frac{2i+1}{d_{e}}}})  $


In above, $ pos = 0,...,d_{l} $ and $ i = 0,...,\lfloor d_{e}/2 \rfloor$ where, $ d_{l} $ is the length of input tokens and $ d_{e} $ is size of embeddings. In this example we have:

In [17]:
d_l = input_embeddings.size(0)
d_e = input_embeddings.size(1)
print(f"d_l is: {d_l} and d_e is: {d_e}")

d_l is: 5 and d_e is: 6


In [18]:
import math
def calculate_pe(d_l,d_e,n):
    pe = torch.zeros(d_l,d_e)
    for pos in range(d_l):
        for i in range(d_e //2):
            denominator = pos / (n**(2*i/d_e)) 

            pe[pos, 2*i] = math.sin(denominator) 
            pe[pos, 2*i+1] = math.cos(denominator)
    return pe

position_embeddings = calculate_pe(d_l,d_e,10000)
print(position_embeddings.shape)

torch.Size([5, 6])


If you see in the equation, for $ 2i $ and $ 2i+1 $ we have a same denominator which is $ 10000^{\frac{2i}{d_{embed}}} $. Therefore, we can pre-calculate them in order to improve computation performance in large sequences. Moreover, we can reformulate the denominator as $ e^{- \frac{2i \; log(10000)}{d_{embed}}} $. For the sake of simplicity, we will not go through this implementation, however, you can refer to [2] for more details.  

Finally, we add the position embedding to the input embedding. Since they can be the same size, so that the two can be summed. See figure 1. 

<center><figure><img src="imgs/positional_encoding.png" alt="drawing" width="300"/><figcaption>Fig. 1: Positional Encoding.</figcaption></figure></center>  

In [13]:
embeddings = input_embeddings + position_embeddings

In [46]:
# Pytorch implementation
# position = torch.arange(seq_length).unsqueeze(1)
# div_term = torch.exp(torch.arange(0, d_model, 2) * (-math.log(10000.0) / d_model))
# print(div_term)
# pe = torch.zeros(seq_length, 1, d_model)
# print(pe.shape)
# pe[:, 0, 0::2] = torch.sin(position * div_term)
# pe[:, 0, 1::2] = torch.cos(position * div_term)

First we should generate attention scores, which simply is the dot product of each embedding vector with other embedding vectors. Dot product is used as a similarity function.

That is $ Attention Scores \in \mathbb{R}^{d_{l}\times d_{l}} $ \
as mentioned above $ d_{l} $ is the number of input tokens (i.e., words), here $ d_{l} = 5 $   

In [21]:
attn_scores = torch.matmul(embeddings, embeddings.T) 
print("attention scores are: ", attn_scores)

attention scores are:  tensor([[6.0334, 5.4477, 4.7140, 4.9825, 4.0555],
        [5.4477, 6.3150, 5.6911, 5.1074, 3.2900],
        [4.7140, 5.6911, 7.2858, 6.6659, 3.7172],
        [4.9825, 5.1074, 6.6659, 8.0217, 5.1145],
        [4.0555, 3.2900, 3.7172, 5.1145, 4.4839]])


Then, we normalize the attention scores using softmax function. The main goal behind the normalization is to obtain attention weights that sum up to 1. 

$ (3) \; \sigma(\mathbf{z})_{i} = \frac{e^{z_{i}}}{\sum_{j=1}^{K} e^{z_{j}}}, for \; i = 1 ,...,K \; and \; \mathbf{z} \in \mathbb{R}^{K} $

In [22]:
attn_weights = torch.softmax(attn_scores, dim=1)
print("attention weights are: ", attn_weights)
print("Sums of all rows are: ",attn_weights.sum(dim=1))

attention weights are:  tensor([[0.4325, 0.2408, 0.1156, 0.1512, 0.0598],
        [0.1824, 0.4341, 0.2326, 0.1298, 0.0211],
        [0.0414, 0.1100, 0.5418, 0.2915, 0.0153],
        [0.0338, 0.0383, 0.1822, 0.7070, 0.0386],
        [0.1516, 0.0705, 0.1081, 0.4371, 0.2327]])
Sums of all rows are:  tensor([1.0000, 1.0000, 1.0000, 1.0000, 1.0000])


In [23]:
context_vectors = torch.matmul(attn_weights, embeddings)  
print("context_vectors are: ",context_vectors) # 5x6

context_vectors are:  tensor([[ 0.4969,  0.7521,  0.6448,  1.4542,  0.5668,  1.3252],
        [ 0.8145,  0.6362,  0.6052,  1.4879,  0.3682,  1.3858],
        [ 0.8516, -0.0091,  0.4480,  1.6620,  0.4033,  1.6351],
        [ 0.5378, -0.2659,  0.7174,  1.5309,  0.7181,  1.8102],
        [ 0.2635,  0.0893,  0.7225,  1.4187,  0.6513,  1.6058]])


As you can see, context vectors are the same size as our inputs. In other ways, we simply modified the embeddings to reflect the attention to other tokens as well.

# Self-Attention

There are several ways to implement a self-attention layer. The original implementation  introduced in the original paper is called <em> "Scaled Dot-Product Attention" </em> as depicted in figure 2.


Why <em> Scaled </em> dot-product? When scaling up the embedding dimension, which is typically greater than thousand for GPT-like LLMs, large dot products can result in very small gradients during backpropagation due to the softmax function applied to them. As dot products increase, the softmax function behaves more like a step function, resulting in gradients nearing zero. These small gradients can drastically slow down learning. In scaled dot-product attention, the dot products are scaled by the size of the embedding vectors so that we don't get too many large numbers during training. 
<center><figure><img src="imgs/scaled-dot-product.png" alt="drawing" width="300"/><figcaption>Fig. 2: Scaled Dot-Product Attention.</figcaption></figure></center>    

The scaled-dot-product is obtained using the following equation: 

$ (4) \; Attention(Q,K,V) =  softmax(\frac{QK^{T}}{\sqrt{d_{k}}})V$   

Where, Q, K, and V are called query, key, and value, respectively. 

# What are Query (Q), Key (K) and Value (V)? 

In attention mechanisms, we use terms "key," "query," and "value" which come from information retrieval and databases. They help us store, search, and get information efficiently.

Think of a "query" like a search term you put into a database. It's what the model is currently focusing on or trying to understand, like a word in a sentence. The query helps the model figure out how much attention to give to other parts of the input.

A "key" is like an index in a database used for searching. Each item in the input sequence, such as each word in a sentence, has a key. These keys are matched with the query to find relevant information.

The "value" in this context is similar to the value in a key-value pair in a database. It represents the actual content or representation of the input items. Once the model determines which keys (and thus which parts of the input) are most relevant to the query (the current focus item), it retrieves the corresponding values. 

Let's see figure 2 in code. First, we write a function to implement a scaled dot-product. 

In [28]:
from math import sqrt
def scaled_dot_product_attention(Q, K, V):
    dim_k = K.size(-1)
    print(dim_k)
    attn_scores = torch.matmul(Q, K.T)
    attn_weights = torch.softmax(attn_scores / sqrt(dim_k),dim=-1)
    return torch.matmul(attn_weights,V)

Then, we create self-attention class as follows. $W_{q}, W_{k}, W_{v}$ are weight matrices. We will use <code>nn.Linear</code> to take advantage of its weight initialization scheme and set <code>bias = False</code> to perform matrix multiplication as well.

In [27]:
class SelfAttention(nn.Module):
    def __init__(self, embed_dim, head_dim):
        super().__init__()
        self.W_q = nn.Linear(embed_dim, head_dim, bias=False)
        self.W_k = nn.Linear(embed_dim, head_dim, bias=False)
        self.W_v = nn.Linear(embed_dim, head_dim, bias=False)
        
    def forward(self, x):
        keys = self.W_k(x)
        queries = self.W_q(x)
        values = self.W_v(x)        
  
        context_vector = scaled_dot_product_attention(queries, keys, values)
        return context_vector    

# Multi-head attention
As we saw, the self-attention mechanism employs three independent linear transformations on each embedding to produce the query, key, and value vectors. Each projection has  its own set of trainable parameters, so that the model, especially the self-attention layer, can attend to various semantic features within the sequence and  learn to produce "good" context vectors.		 	 	 		
			

Additionally, it would be advantageous to incorporate multiple sets of linear projections, referred to as  “attention head”. But why several attention heads? The reason is the softmax function of a single head focuses on one similarity aspect. By employing several heads, the model simultaneously focuses on multiple aspects of similarity.

$(5) \; MultiHead(Q,K,V) =  Concat(head_{1}, ..., head_{h})W^{O} $ 

where, 

$ head_{i} = Attention(QW_{i}^{Q},KW_{i}^{K},VW_{i}^{V})$ and, \
$ W_{i}^{Q} \in \mathbb{R}^{d_{e}\times d_{k}}, W_{i}^{K} \in \mathbb{R}^{d_{e} \times d_{k}}, W_{i}^{V} \in \mathbb{R}^{d_{e} \times d_{v}} and \; W^{O} \in \mathbb{R}^{hd_{v} \times d_{e}} $ are weight matrices.

Also, $ d_{k} = d_{v} = d_{e} / h $ where, h is the number of heads

These matrices transform input data into queries, keys, and values, which are crucial components of the attention mechanism. As the model is exposed to more data during training, it adjusts these trainable weights.

<center><figure><img src="imgs/multi-head-attention.png" alt="drawing" width="350"/><figcaption>Fig. 3: Multi-head Attention.</figcaption></figure></center>    

Now that we have implemented a self-attention mechanism, let's move forward and implement a multi-head attention mechanism according to figure 3.

In [30]:
d_e = embeddings.shape[1]
num_heads = 2
d_h = d_e // num_heads
print(f"Embedding size (d_e) is: {d_e}, head dimension (d_h) is: {d_h} and number of heads are: {num_heads}")

Embedding size (d_e) is: 6, head dimension (d_h) is: 3 and number of heads are: 2


In [31]:
class MultiHeadAttention(nn.Module):
    def __init__(self, embed_size, num_heads, head_dim):
        super().__init__()
        self.heads = nn.ModuleList([SelfAttention(embed_size, head_dim) for _ in range(num_heads)])
        self.output_linear = nn.Linear(embed_size, embed_size)
        
    def forward(self, inputs):
        x = torch.cat([head(inputs) for head in self.heads], dim=-1)
        x = self.output_linear(x)
        return x   

Please note that the final linear layer is used to produce a tensor of the same size as our input tensor (i.e., 5x6 in our example). Let's check it out!

In [32]:
mult_head_attn = MultiHeadAttention(d_e,num_heads,d_h)
attn_output = mult_head_attn(embeddings)
print(attn_output.shape)

3
3
torch.Size([5, 6])


As you can see it generates an output tensor with the expected size.

If we look at the model architecture depicted in figure 3, there is a fully connected feed-forward network, which is applied to each position separately and identically. This consists of two linear transformations with a ReLU activation in between.

$(5) \; FFN(x) = max(0, xW_{1} + b_{1})W_{2} + b_{2} $

<center><figure><img src="imgs/transformer architecture.png" alt="drawing" width="400"/><figcaption>Fig. 4: Transformer architecture.</figcaption></figure></center> 

In [95]:
ff_hidden_size = 4 * embed_size 

In [110]:
class FeedForwardLayer(nn.Module):
    def __init__(self, embed_size, ff_hidden_size):
        super().__init__()
        self.x1 = nn.Linear(embed_size, ff_hidden_size)
        self.x2 = nn.Linear(ff_hidden_size, embed_size)
        self.relu = nn.ReLU()
    
    def forward(self,x):
        x = self.x1(x)
        x = self.relu(x)
        x = self.x2(x)
        return x
        

Moreover, a residual connection is employed around each of the two sub-layers, followed by layer normalization. That is, the output of each sub-layer is
$ LayerNorm(x + Sublayer(x)) $, where $ Sublayer(x) $ is the function implemented by the sub-layer itself [1].

Note: As a rule of thumb, the size of first layer is four times the size of the embeddings. 

In [111]:
feed_forward = FeedForwardLayer(embed_size, ff_hidden_size)
ff_out = feed_forward(embeddings)
ff_out.shape

torch.Size([5, 6])

In [112]:
class Encoder(nn.Module):
    def __init__(self, embed_size, num_heads, head_dim, ff_hidden_size):
        super().__init__()
        self.norm1 = nn.LayerNorm(embed_size)
        self.norm2 = nn.LayerNorm(embed_size)
        self.multi_head_attn = MultiHeadAttention(embed_size,num_heads,head_dim)
        self.feed_forward = FeedForwardLayer(embed_size,ff_hidden_size)
        
    def forward(self,x):
        x = self.norm1(x + self.multi_head_attn(x))
        x = self.norm2( x + self.feed_forward(x))
        return x

Now, we can easily add the pieces together and build the encoder part as shown in the left side of figure 4. 

In [113]:
encoder = Encoder(embed_size,num_heads,head_dim,ff_hidden_size)
encoder_output = encoder(embeddings)
encoder_output.shape

torch.Size([5, 6])

In the next article, I will implement the decoder part. 

# References: 

[1] Ashish Vaswan, Noam Shazeer, Niki Parmar, Jakob Uszkoreit, Llion Jones, Aidan N. Gomez, Łukasz Kaiser, Illia Polosukhin. "Attention Is All You Need". 	arXiv:1706.03762v7. 

[2] https://pytorch.org/tutorials/beginner/transformer_tutorial.html